In [2]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [13]:
bucket_name = 'bankapplication162023'

In [15]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'

output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print (output_path)

s3://bankapplication162023/xgboost-as-a-built-in-algo/output


In [16]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [17]:
import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [19]:
import os

# Upload the training data to S3
train_data.to_csv('train.csv', index=False, header=False)
s3 = boto3.Session().resource('s3')
s3.Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')

In [20]:
from sagemaker.inputs import TrainingInput

s3_input_train = TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/train', content_type='csv')

In [22]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
s3 = boto3.Session().resource('s3')
s3.Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

In [23]:
s3_input_test = TrainingInput(s3_data=f's3://{bucket_name}/{prefix}/test', content_type='csv')

In [25]:
from sagemaker.image_uris import retrieve

container = retrieve("xgboost", boto3.Session().region_name, version="1.0-1")

In [26]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [28]:
from sagemaker.estimator import Estimator

estimator = Estimator(image_uri=container,  # Use the image URI
                      hyperparameters=hyperparameters,
                      role=sagemaker.get_execution_role(),
                      instance_count=1, 
                      instance_type='ml.m5.2xlarge', 
                      volume_size=5,  # 5 GB
                      output_path=output_path,
                      use_spot_instances=True,
                      max_run=300,
                      max_wait=600)

NOTEBOOK_METADATA_FILE detected but failed to get valid domain and user from it.


In [29]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-10-16-09-40-24-219


2023-10-16 09:40:24 Starting - Starting the training job...
2023-10-16 09:40:39 Starting - Preparing the instances for training...
2023-10-16 09:41:20 Downloading - Downloading input data......
2023-10-16 09:42:11 Training - Training image download completed. Training in progress..[2023-10-16 09:42:19.140 ip-10-0-68-130.ap-south-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node tr

UnexpectedStatusException: Error for Training job sagemaker-xgboost-2023-10-16-09-40-24-219: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_xgboost_container/algorithm_mode/train.py", line 226, in train_job
    verbose_eval=False)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/training.py", line 209, in train
    xgb_model=xgb_model, callbacks=callbacks)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/training.py", line 30, in _train_internal
    bst = Booster(params, [dtrain] + [d[0] for d in evals])
  File "/miniconda3/lib/python3.7/site-packages/xgboost/core.py", line 1028, in __init__
    self._validate_features(d)
  File "/miniconda3/lib/python3.7/site-packages/xgboost/core.py", line 1854, in _validate_features
    data.feature_names))
ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15